In [301]:
import pandas as pd
import pycountry
import PyPDF2 
import textract
import re

In [517]:
# Initialize list sequence to store names of Countries
countries = [
    'Afghanistan','Albania','Algeria','Angola','Antigua and Barbuda',
    'Argentina','Armenia','Aruba','Australia','Austria','Azerbaijan',
    'The Bahamas','Bahrain','Bangladesh','Barbados','Belarus','Belgium', 
    'Belize','Benin','Bhutan','Bolivia','Bosnia and Herzegovina',
    'Botswana','Brazil','Brunei','Bulgaria','Burkina Faso','Burma',
    'Burundi','Cabo Verde','Cambodia','Cameroon','Canada','Central African Republic',
    'Chad','Chile','China','Colombia','Comoros','Congo, Democratic Republic of the',
    'Congo, Republic of the','Costa Rica',"Cote d'Ivoire",'Croatia','Cuba','Curaçao',
    'Cyprus','Czech Republic', 'Denmark','Djibouti', 'Dominican Republic','Ecuador',
    'Egypt','El Salvador','Equatorial Guinea', 'Eritrea','Estonia','Eswatini','Ethiopia',
    'Fiji', 'Finland', 'France','Gabon','The Gambia','Georgia','Germany','Ghana','Greece',
    'Guatemala','Guinea','Guinea-Bissau','Guyana','Haiti','Honduras','Hong Kong','Hungary',
    'Iceland','India','Indonesia','Iran','Iraq','Ireland','Israel','Italy','Jamaica','Japan',
    'Jordan','Kazakhstan','Kenya',"Korea, Democratic People's Republic of",'Korea, Republic of',
    'Kosovo','Kuwait','Kyrgyz Republic',"Laos",'Latvia','Lebanon','Lesotho','Liberia','Lithuania',
    'Luxembourg','Macau','Macedonia','Madagascar','Malawi','Malaysia','Maldives','Mali','Malta',
    'Marshall Islands','Mauritania','Mauritius','Mexico','Micronesia, Federated States of',
    'Moldova','Mongolia','Montenegro','Morocco','Mozambique','Namibia','Nepal','Netherlands',
    'New Zealand','Nicaragua','Niger','Nigeria','Norway','Oman','Pakistan','Palau','Panama',
    'Papua Guinea New','Paraguay','Peru','Philippines','Poland','Portugal','Qatar','Romania',
    'Russia','Rwanda','St. Lucia','St. Vincent and the Grenadines','Saudi Arabia','Senegal','Serbia',
    'Seychelles','Sierra Leone','Singapore','Slovakia','Slovenia','Solomon Islands','South Africa',
    'South Sudan','Spain','Sri Lanka','Sudan','Suriname','Sweden','Switzerland','Syria','Taiwan',
    'Tajikistan','Tanzania','Thailand','Timor-Leste','Togo','Tonga','Trinidad and Tobago','Tunisia',
    'Turkey','Turkmenistan','Uganda','Ukraine','United Arab Emirates','United Kingdom',
    'United States of America','Uruguay','Uzbekistan','Venezuela', 'Vietnam','Zambia','Zimbabwe',
    'Libya','St. Maarten','Somalia','Yemen' #Libya, St. Marteen, Somalia and Yemen are Special Cases
]


In [303]:
len(countries)

187

In [305]:
# Function to Parse the pdf file
def parse_file(filename): 
    # Read the file.
    pdfFileObj = open(filename,'rb')
    # The pdfReader variable is the readable object that will be parsed.
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    #Parse through all the pages.
    num_pages = pdfReader.numPages
    count = 0
    text = ""
    # The while loop will read each page.
    while count < num_pages:
        pageObj = pdfReader.getPage(count)
        count +=1
        text += pageObj.extractText()
    # This if statement exists to check if the above library returned words. It's done because PyPDF2 cannot read scanned files.
    if text != "":
        text = text
    # If the above returns as False, run the OCR library textract to #convert scanned/image based PDF files into text.
    else:
        text = textract.process(fileurl, method='tesseract', language='eng')
    return text

text = parse_file('2018-TIP-Report.pdf')
    

In [306]:
# Function to Clean the parsed pdf
def clean_text(text):
    text = text.strip().replace("\n", "").replace("˜", "fi").replace("˚", "ft").replace("Sincerely,iiiiii","")\
                .replace("Trafiick","Traffick").replace("™", "'").replace("trafiick","traffick")\
                .replace('ﬁprevalence rateﬂ','"prevalence rate"').replace("˚ight","fight")\
                .replace("/", "").replace("  REPUBLIC OF THE:", " REPUBLIC OF THE:")\
                .replace("DEMOCRATIC PEOPLE'S  REPUBLIC OF", "DEMOCRATIC PEOPLE'S REPUBLIC OF")\
                .replace("PAPUA NEW GUINEA:", "PAPUA GUINEA NEW:").replace("⁄", "").replace("\\","")\
                .replace("ST. VINCENT AND  THE GRENADINES", "ST. VINCENT AND THE GRENADINES")\
                .replace("UNITED STATES  OF AMERICA", "UNITED STATES OF AMERICA")\
                .replace("461SPECIAL CASE: LIBYA", " ").replace("462SPECIAL CASE: LIBYA", " ")\
                .replace("463SPECIAL CASE: ST. MAARTEN", " ").replace("464SPECIAL CASE: SOMALIA", " ")\
                .replace("465SPECIAL CASE: SOMALIA", " ").replace("466SPECIAL CASE: YEMEN", " ")\
                .replace("467SPECIAL CASE: YEMEN", " ").replace("468SPECIAL CASE: YEMEN", " ")
    return text 

cleaned_text = clean_text(text)


In [307]:
# Function to Extract each country data from text
def extract_info(countries_list,text):
    # Create list to query info for each country
    countries_up = []
    for i in countries:
        if i in ["Libya", 'St. Maarten', 'Somalia', 'Yemen']:
            countries_up.append("SPECIAL CASE: "+i.upper())
        else: 
            countries_up.append(i.upper()+":")        
    new_list = []
    a = text.split(countries_up[0])
    for i in range(len(countries_up)-1):
        a = a[1].split(countries_up[i+1])
        new_list.append(a[0])
        if i+1 == len(countries_up)-1:
            new_list.append(a[1].split("469RELEVANT INTERNATIONAL CONVENTIONS")[0])
    return new_list

country_info = extract_info(countries,cleaned_text)
len(country_info)


187

In [7]:
# Function to Extract each country data from text
def extract_info(countries_list,text):
    # Create list to query info for each country
    countries_up = []
    for i in countries:
        if i in ["Libya", 'St. Maarten', 'Somalia', 'Yemen']:
            countries_up.append("SPECIAL CASE: "+i.upper())
        else: 
            countries_up.append(i.upper()+":")
    new_list = []
    a = text.split(countries_up[0])
    for i in range(len(countries_up)-1):
        a = a[1].split(countries_up[i+1])
        new_list.append(a[0])
        if i+1 == len(countries_up)-1:
            new_list.append(a[1].split("469RELEVANT INTERNATIONAL CONVENTIONS")[0])
    return new_list

country_info = extract_info(countries,cleaned_text)
len(country_info)


187

In [308]:
country_dict = {i.upper():[{j:k}] for i,j,k in zip(countries, ["INTRO"]*187, country_info)}
special = [s.upper() for s in ['Libya','St. Maarten','Somalia','Yemen']]
special_country_dict = {s:country_dict[s] for s in special}
print(len(country_dict.values()))
print(len(special_country_dict))

187
4


In [585]:
def removekey(d,delete_items):
    r = dict(d)
    for key in delete_items: 
        del r[key]
    return r

normal_country_dict = removekey(country_dict, special)
print(len(normal_country_dict.values()))
normal_countr = list(normal_country_dict.keys())

183


In [311]:
def append_data(country,i,data):
    data[0]["INTRO"] = data[0]["INTRO"].split(i)[1]
    data.append({"TIER":i})
    intro = data[0]["INTRO"].split(f"RECOMMENDATIONS FOR {country}")
    data[0]["INTRO"] = intro[0]
    recommend = intro[1].split("PROSECUTION")
    data.append({"RECOMMENDATIONS":recommend[0]})
    prosecute = recommend[1].split("PROTECTION")
    data.append({"PROSECUTIONS":prosecute[0]})
    protect = prosecute[1].split("PREVENTION")
    data.append({"PROTECTION":protect[0]})
    prevent = protect[1].split("TRAFFICKING PROFILE")
    data.append({"PREVENTION":prevent[0]})
    data.append({"TRAFFICKING PROFILE":prevent[1]})
    return data

In [310]:
def portion_data(country):
    data = normal_country_dict[country]
    tier = ["TIER 1", "TIER 2", "TIER 3"]
    for i in tier:
        length = len(data[0]["INTRO"].split(i))       
        if "TIER 2 WATCH LIST" in data[0]["INTRO"]:
            i = "TIER 2 WATCH LIST"
            append_data(country,i,data)
        elif length == 2 and "TIER 2 WATCH LIST" not in data[0]["INTRO"]: 
            append_data(country,i,data)            
    return data,country_dict


In [314]:
x = countries[:183]  #183
new_normal_dict = [{i.upper():portion_data(i.upper())[0]} for i in x]


In [555]:
def get_country_special(country):
    new_special = []
    intro = special_country_dict[country][0]["INTRO"].split("GOVERNMENT EFFORTS")
    new_special.append({"INTRO":intro[0]})
    new_special.append({"TIER": "SPECIAL CASE"})
    traffick = intro[1].split("TRAFFICKING PROFILE")
    new_special.append({"GOVERNMENT EFFORTS":traffick[0]})
    new_special.append({"TRAFFICKING PROFILE":traffick[1]})
    return new_special


In [569]:
special = [i.upper() for i in ["Libya", 'St. Maarten', 'Somalia', 'Yemen']]
new_special_dict = [{i:get_country_special(i)} for i in special]

In [562]:
def get_traffic_profile(country_list,country_dict):
    return [{a:list(country_dict[country_list.index(a)][a][-1].values())[0]} for a,b in zip(country_list, country_dict)]


In [575]:
normal_trafficking_profile = get_traffic_profile(normal_countr,new_normal_dict)
special_trafficking_profile = get_traffic_profile(special,new_special_dict)


In [581]:
special_trafficking_profile = get_traffic_profile(special,new_special_dict)
trafficking_profile = normal_trafficking_profile.copy()
trafficking_profile.extend(special_trafficking_profile)


In [589]:
new_normal_countr = normal_countr.copy()
new_normal_countr.extend(special)


In [565]:
changed_countries = [
    'Afghanistan','Albania','Algeria','Angola','Antigua and Barbuda',
    'Argentina','Armenia','Aruba','Australia','Austria','Azerbaijan',
    'The Bahamas','Bahrain','Bangladesh','Barbados','Belarus','Belgium', 
    'Belize','Benin','Bhutan','Bolivia','Bosnia and Herzegovina',
    'Botswana','Brazil','Brunei','Bulgaria','Burkina Faso','Burma',
    'Burundi','Cabo Verde','Cambodia','Cameroon','Canada','Central African Republic',
    'Chad','Chile','China','Colombia','Comoros','Democratic',
    'Republic of The Congo','Costa Rica',"Cote d'Ivoire",'Croatia','Cuba','Curaçao',
    'Cyprus','Czech Republic', 'Denmark','Djibouti', 'Dominican Republic','Ecuador',
    'Egypt','El Salvador','Equatorial Guinea', 'Eritrea','Estonia','Eswatini','Ethiopia',
    'Fiji', 'Finland', 'France','Gabon','The Gambia','Georgia','Germany','Ghana','Greece',
    'Guatemala','Guinea','Guinea-Bissau','Guyana','Haiti','Honduras','Hong Kong','Hungary',
    'Iceland','India','Indonesia','Iran','Iraq','Ireland','Israel','Italy','Jamaica','Japan',
    'Jordan','Kazakhstan','Kenya',"North Korea",'South Korea',
    'Kosovo','Kuwait','Kyrgyz Republic',"Laos",'Latvia','Lebanon','Lesotho','Liberia','Lithuania',
    'Luxembourg','Macau','Macedonia','Madagascar','Malawi','Malaysia','Maldives','Mali','Malta',
    'Marshall Islands','Mauritania','Mauritius','Mexico','Micronesia','Moldova','Mongolia',
    'Montenegro','Morocco','Mozambique','Namibia','Nepal','Netherlands','New Zealand','Nicaragua',
    'Niger','Nigeria','Norway','Oman','Pakistan','Palau','Panama','Papua Guinea New','Paraguay',
    'Peru','Philippines','Poland','Portugal','Qatar','Romania','Russia','Rwanda','St. Lucia',
    'St. Vincent and the Grenadines','Saudi Arabia','Senegal','Serbia','Seychelles','Sierra Leone',
    'Singapore','Slovakia','Slovenia','Solomon Islands','South Africa','South Sudan','Spain',
    'Sri Lanka','Sudan','Suriname','Sweden','Switzerland','Syria','Taiwan','Tajikistan','Tanzania',
    'Thailand','Timor-Leste','Togo','Tonga','Trinidad and Tobago','Tunisia','Turkey','Turkmenistan',
    'Uganda','Ukraine','United Arab Emirates','United Kingdom','United States','Uruguay',
    'Uzbekistan','Venezuela', 'Vietnam','Zambia','Zimbabwe','Libya','St. Maarten','Somalia','Yemen' 
    #Libya, St. Marteen, Somalia and Yemen are Special Cases
]


In [596]:
def get_dataframe(countries,country_list,trafficking_profile):
    x = 0
    j = {
        "Base Country":[],
        "Relation Country":[],
        "Men": [],
        "Women":[],
        "Children":[],
        "Sex Trafficking":[],
        "Forced Labor":[],
        "Debt Bondage":[],
        "Description":[]
        }
    for country in country_list:
        a = list(trafficking_profile[x].values())[0].split(".")
        a = a[:len(a)-1]
        strip_country = countries.copy()
        d = strip_country.pop(country_list.index(country))   
        for b in a:
            for i in strip_country:
                if i in b:
                    if "men" in b or "women" in b or "boy" in b or "girl" in b or "child" in b or "Men" in b or "Women" in b or "Boy" in b or "Girl" in b or "Child" in b:
                        j["Relation Country"].append(i)
                        j["Base Country"].append(d)
                        j["Description"].append(b)
                        if "women" in b or "Women" in b:
                            j["Women"].append(1)
                        else:
                            j["Women"].append(0)
                        if "men" in b or "Men" in b and "women" not in b and "Women" not in b:
                            j["Men"].append(1)
                        else:
                            j["Men"].append(0)
                        if "boy" in b or "girl" in b or "child" in b or "Boy" in b or "Girl" in b or "Child" in b:
                            j["Children"].append(1)
                        else:
                            j["Children"].append(0)
                        if "debt" in b or "Debt" in b:
                            j["Debt Bondage"].append(1)
                        else:
                            j["Debt Bondage"].append(0)
                        if "sex" in b or "Sex" in b:
                            j["Sex Trafficking"].append(1)
                        else:
                            j["Sex Trafficking"].append(0)
                        if "force" in b or "Forced" in b:
                            j["Forced Labor"].append(1)
                        else:
                            j["Forced Labor"].append(0)
        x += 1
    df = pd.DataFrame.from_dict(j, orient='index').transpose()
    return df              


In [598]:
a = get_dataframe(changed_countries,new_normal_countr,trafficking_profile)

In [ ]:
a